In [ ]:
import pandas as pd
import numpy as np
import xlwings as xw

In [ ]:
df = pd.read_excel("option_chain_bank.xlsx")

In [ ]:
## 4 main object -> App, Book, Sheet, Range, chart , shape
wb = xw.Book()
ws = wb.sheets["Sheet1"]
#checks that wb.sheets[0] equals ws
print(ws == wb.sheets[0])
ws.name = "first_sheet"

In [ ]:
expiry_date = "2023-03-23"
lower_sp = 38000
upper_sp = 43000

req_cols = ['strikePrice','expiryDate','openInterest','pchangeinOpenInterest','changeinOpenInterest','instrumentType']
df_filter = df[req_cols]
df_total = df.groupby(['expiryDate','instrumentType']).agg(
    {
    'openInterest': sum 
    }
    ).reset_index()
df_filter = df_filter.append(df_total)
df_filter['strikePrice'].fillna(0, inplace=True)

In [ ]:

df_filter = (
    df_filter.query('instrumentType == "CE"')
    .merge(df_filter.query('instrumentType == "PE"'),
    how = 'inner',
    on = ['strikePrice','expiryDate'],
    suffixes=('_CE','_PE')
    )
    .drop(['instrumentType_CE','instrumentType_PE'],axis = 1)
    .query(f'expiryDate == "{expiry_date}"')
    .query(f'(strikePrice == 0) | ((strikePrice >={lower_sp}) & (strikePrice <= {upper_sp}))' )
)
df_filter.sort_values('strikePrice',ascending=True,inplace=True)

df_filter.reset_index(drop=True, inplace=True)
df_filter['overall_pchange_CE'] = (df_filter['openInterest_CE']/ df_filter['openInterest_CE'][0]) * 100
df_filter['overall_pchange_PE'] = (df_filter['openInterest_PE']/ df_filter['openInterest_PE'][0]) * 100
df_filter = df_filter[['expiryDate', 'openInterest_CE','changeinOpenInterest_CE',
       'pchangeinOpenInterest_CE','overall_pchange_CE' ,
       'strikePrice','overall_pchange_PE','pchangeinOpenInterest_PE',
       'openInterest_PE', 'changeinOpenInterest_PE'
       ]]
df_filter = df_filter.round(2)
df_filter= df_filter.loc[1:,]

In [ ]:
ws.clear_contents()
ws.range("A1").options(index=False).value = df_filter